In [1]:
import pandas as pd

In [2]:
date_format = '%m/%d/%y'
caphis = pd.read_csv("data.csv",parse_dates=["DATE"],index_col="DATE",date_format=date_format)
caphistory = pd.DataFrame()

In [3]:
caphis.dropna(subset=["SERIAL_NUMBER"],inplace=True)

In [ ]:
caphis.head(5)

In [5]:
caphis =  caphis[["CHARGE_FULL","CHARGE_FULL_DESIGN"]]

In [6]:
caphis["CHARGE_FULL"] = (caphis["CHARGE_FULL"]/1000).astype(int)
caphis["CHARGE_FULL_DESIGN"] = (caphis["CHARGE_FULL_DESIGN"]/1000).astype(int)

In [ ]:
caphis

In [8]:
curr_date=pd.to_datetime(caphis.index[-1])
week_d=curr_date+pd.DateOffset(weeks=1)
st_l = [];en_l = []; ch_l = []

In [9]:
while True:
    st_l.append(curr_date)
    en_l.append(week_d)
    ch_l.append(caphis.loc[(caphis.index>=curr_date)&(caphis.index<=week_d)]["CHARGE_FULL"].mean().astype(int))
    curr_date = curr_date+pd.DateOffset(weeks=1)
    week_d = week_d+pd.DateOffset(weeks=1)
    if(week_d>=caphis.index[0]+pd.DateOffset(days=1)):
        break

In [11]:
if curr_date<caphis.index[0]:
    week_d = caphis.index[0]
    st_l.append(curr_date)
    en_l.append(week_d)
    ch_l.append(caphis.loc[(caphis.index>=curr_date)&(caphis.index<=week_d)]["CHARGE_FULL"].mean().astype(int))
    

In [12]:
caphistory["START DATE"] = pd.Series(st_l)
caphistory["END_DATE"] = pd.Series(en_l)
caphistory["CHARGE_FULL (in mWh)"] = pd.Series(ch_l)
caphistory["CHARGE_FULL_DESIGN (in mWh)"] = caphis["CHARGE_FULL_DESIGN"][0]

In [ ]:
caphistory

In [14]:
caphistory.to_csv("battery-capacity-history.csv")

## Battery capacity history based on Group BY command

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("final.csv")

In [3]:
df.isnull().sum()

DATE                  0
TIME                  0
STATE                 0
DEVTYPE               0
NAME                  0
SOURCE                0
STATUS                0
PRESENT               0
TECHNOLOGY            0
CYCLE_COUNT           0
VOLTAGE_MIN_DESIGN    0
VOLTAGE_NOW           0
CURRENT_NOW           0
CHARGE_FULL_DESIGN    0
CHARGE_FULL           0
CHARGE_NOW            0
CAPACITY              0
CAPACITY_LEVEL        0
MODEL_NAME            0
MANUFACTURER          0
SERIAL_NUMBER         0
dtype: int64

In [4]:
df["DATETIME"] = df["DATE"]+" "+df["TIME"]
df["DATETIME"] = pd.to_datetime(df["DATETIME"],format="%m/%d/%y %H:%M:%S")

In [5]:
df1 = df[["DATETIME","CHARGE_FULL", "CHARGE_FULL_DESIGN"]].copy()

In [6]:
df1

,DATETIME,CHARGE_FULL,CHARGE_FULL_DESIGN
0,2025-02-19 20:51:46,1724000,3684000
1,2025-02-19 18:20:56,1724000,3684000
2,2025-02-19 16:55:21,1724000,3684000
3,2025-02-19 16:43:10,1724000,3684000
4,2025-02-19 16:37:29,1724000,3684000
...,...,...,...
3195,2024-10-03 21:57:38,1403000,3684000
3196,2024-10-03 20:53:34,1403000,3684000
3197,2024-10-03 19:59:00,1403000,3684000
3198,2024-10-03 18:40:25,1403000,3684000


In [7]:
prev_charge_full = df1["CHARGE_FULL"].iloc[0].copy()
grp = 1
for i in range(len(df1)):
    if df1["CHARGE_FULL"].iloc[i] == prev_charge_full:
        df1.loc[i,"GROUP"] = grp
    else:
        prev_charge_full = df1["CHARGE_FULL"].iloc[i]
        grp += 1

In [8]:
df1_gr = pd.DataFrame()

In [9]:
df1_gr["START_DATE"] = df1.groupby("GROUP").agg({"DATETIME":"first"})
df1_gr["END_DATE"] = df1.groupby("GROUP").agg({"DATETIME":"last"})
df1_gr["CHARGE_FULL(mAh)"] = df1.groupby("GROUP").agg({"CHARGE_FULL":"first"})
df1_gr["CHARGE_FULL_DESIGN(mAh)"] = df1.groupby("GROUP").agg({"CHARGE_FULL_DESIGN":"first"})
df1_gr.reset_index(drop=True, inplace=True)

In [10]:
df1

,DATETIME,CHARGE_FULL,CHARGE_FULL_DESIGN,GROUP
0,2025-02-19 20:51:46,1724000,3684000,1.0
1,2025-02-19 18:20:56,1724000,3684000,1.0
2,2025-02-19 16:55:21,1724000,3684000,1.0
3,2025-02-19 16:43:10,1724000,3684000,1.0
4,2025-02-19 16:37:29,1724000,3684000,1.0
...,...,...,...,...
3195,2024-10-03 21:57:38,1403000,3684000,26.0
3196,2024-10-03 20:53:34,1403000,3684000,26.0
3197,2024-10-03 19:59:00,1403000,3684000,26.0
3198,2024-10-03 18:40:25,1403000,3684000,26.0


In [11]:
df1_gr.to_html("battery-capacity-history.html",index=False)

In [12]:
df1_gr

,START_DATE,END_DATE,CHARGE_FULL(mAh),CHARGE_FULL_DESIGN(mAh)
0,2025-02-19 20:51:46,2025-02-15 09:50:05,1724000,3684000
1,2025-02-13 20:42:47,2025-02-13 09:39:49,1652000,3684000
2,2025-02-12 21:35:50,2025-02-12 08:18:16,1908000,3684000
3,2025-02-11 18:31:38,2025-02-06 17:25:07,1396000,3684000
4,2025-02-05 14:56:42,2025-01-30 08:12:51,884000,3684000
5,2025-01-29 21:50:26,2025-01-18 10:20:13,1140000,3684000
6,2025-01-17 12:15:04,2025-01-08 14:22:25,628000,3684000
7,2025-01-08 14:01:23,2025-01-06 16:55:00,884000,3684000
8,2025-01-06 14:29:31,2024-12-26 18:23:01,912000,3684000
9,2024-12-26 17:56:23,2024-12-22 12:07:32,1168000,3684000


In [13]:
usage_df = df[["DATETIME", "STATE", "STATUS"]].copy()
usage_df["GROUP"] = df1["GROUP"].copy()
usage_df.to_csv("batusageact.csv", index=False)